**GOAL**: check if we can build a sparse basis using seaweeds

- Build graph
- Remove some factors to expose leaves
- Build seaweeds and add them to a basis until they form a complete set
- Keep track of the maximum hamming weight within the basis

In [67]:
include("../code/headers.jl")
include("../code/slim_graphs.jl")
using OffsetArrays

To check whether $v\in span\{\mathcal{B}\}$ where $\mathcal{B}=\{v_1,...,v_k\}$, write the system $B\lambda=v$ and use gaussian elimination. If the system can be put in row echelon form, then the vector lies in the span

In [80]:
function isinspan(v, B)
   if isempty(B) 
        return false
    else
        Baug=[B v]
        gfrrefGF2!(Baug)
        # check for all-zero rows
        for r in size(Baug,1):-1:1
            if iszero(Baug[r,1:end-1]) 
                if iszero(Baug[r,end])
                    continue
                else
                    return false
                end
            else
                return true
            end
        end
    end
end

isinspan (generic function with 2 methods)

In [86]:
function seaweed_basis(nsw, fg, dim, depths, isincore)
    B = falses(0,0)
    cnt = 1
    sw = falses(fg.n)
    for i in 1:nsw
        seed = rand(not_in_core)
        sw .= seaweed_rec(fg, seed, depths, isincore)
        if !isinspan(sw, B)
            B = [B sw]
        end
        # if the whole space is spanned, return
        if size(B,2) == dim
            return B, cnt
        end
    end
    println("Not enough iterations to span the whole space")
    return B, nsw
end

seaweed_basis (generic function with 1 method)

In [62]:
m = factorial(3)*11
R = 0.8/3
r = 1 - 3R
n = round(Int, 3m/(2+r))
@show n
Λ = [0,1-r,r]
K = [0,0,1]
nedges = 3m
lambda,rho = nodes2edges(Λ,K)
b = 1

n = 90


1

In [78]:
nsw = 10
fg = ldpc_graphGF2(n,m,nedges,lambda,rho, accept_multiedges=false)
ff = breduction!(fg, b)
fg.H[:,ff] .= 0
depths = lr(fg)[1]
isincore = (depths .== 0)
not_in_core = findall(.!isincore)
# use gaussian elim to get size of nullspace
H = Array(fg.H)
H, dep = gfrrefGF2!(H)
dim = fg.n - length(dep)

25

In [ ]:
B = 